In [9]:
import pandas as pd
import numpy as np
import json
import time
import matplotlib.pyplot as plt

from scipy.spatial import distance

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot , Tokenizer
from tensorflow.keras.applications.xception import Xception
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input, BatchNormalization, Dropout
from keras import Sequential, Model
from keras.layers import Lambda, Reshape, Dot
from keras.losses import CosineSimilarity
from keras.layers import merge

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
image_embeddings = np.loadtxt("/content/drive/MyDrive/DL Project DS6050/image_embeddings.txt").reshape(27638, 8192)


In [12]:
f = open('/content/drive/MyDrive/DL Project DS6050/text_embs.json')
text_embeddings = json.load(f)
f.close()


In [13]:
text_embeddings = np.array([text_embeddings[str(i)] for i in range(1,27639)])

In [ ]:
#recipe_info = pd.read_csv('/content/drive/MyDrive/DL Project DS6050/recipe_info.csv')

In [ ]:

left_input = Input(shape=(1200,))
right_input = Input(shape=(8192,))
    
premodel = Sequential()
premodel.add(Dense(1200, activation='relu', input_shape=(8192,)))
premodel.add(BatchNormalization())

model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(1200,)))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(500, activation='relu'))


encoded_l = model(left_input)
encoded_r = model(premodel(right_input))


cos_distance = keras.layers.Dot(axes=1, normalize=True)([encoded_l, encoded_r])

cos_distance = Reshape((1,))(cos_distance)
cos_similarity = Lambda(lambda x: 1-x)(cos_distance)

siamese_net = Model([left_input, right_input], [cos_distance])
siamese_net.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
siamese_net.compile(loss="mse",optimizer=optimizer)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 8192)]       0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 1200)]       0           []                               
                                                                                                  
 sequential_6 (Sequential)      (None, 1200)         9836400     ['input_8[0][0]']                
                                                                                                  
 sequential_7 (Sequential)      (None, 500)          1705500     ['input_7[0][0]',                
                                                                  'sequential_6[0][0]']     

In [ ]:
output = np.concatenate((np.ones(27638), np.zeros(27638)))

siamese_net.fit(x=[np.concatenate((text_embeddings, np.concatenate((text_embeddings[2:], text_embeddings[0:2])))),np.concatenate((image_embeddings, image_embeddings))],
                y=output, 
                batch_size=32, 
                epochs=5, 
                validation_split=.2)

Epoch 1/5
1382/1382 [==============================] - 126s 88ms/step - loss: 0.2631 - val_loss: 0.2077
Epoch 2/5
1382/1382 [==============================] - 121s 88ms/step - loss: 0.2276 - val_loss: 0.1510
Epoch 3/5
1382/1382 [==============================] - 121s 88ms/step - loss: 0.2139 - val_loss: 0.1788
Epoch 4/5
1382/1382 [==============================] - 122s 88ms/step - loss: 0.2024 - val_loss: 0.1566
Epoch 5/5
1382/1382 [==============================] - 121s 87ms/step - loss: 0.1932 - val_loss: 0.2145


In [ ]:
siamese_net.predict([text_embeddings,image_embeddings])

array([[0.59649426],
       [0.31678686],
       [0.5264446 ],
       ...,
       [0.5098851 ],
       [0.4507288 ],
       [0.39223075]], dtype=float32)

In [ ]:
siamese_net.predict([text_embeddings[1:7],image_embeddings[20:26]])

array([[0.2977701 ],
       [0.6390219 ],
       [0.6124878 ],
       [0.15426914],
       [0.27225992],
       [0.5616291 ]], dtype=float32)

In [14]:

left_input = Input(shape=(1200,))
right_input = Input(shape=(8192,))
    
premodel = Sequential()
premodel.add(Dense(1200, activation='relu', input_shape=(8192,)))
premodel.add(BatchNormalization())

model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(1200,)))
model.add(Dropout(.35))
model.add(BatchNormalization())
model.add(Dense(900, activation='relu'))
model.add(Dropout(.35))
model.add(BatchNormalization())
model.add(Dense(800, activation='relu'))


encoded_l = model(left_input)
encoded_r = model(premodel(right_input))


cos_distance = keras.layers.Dot(axes=1, normalize=True)([encoded_l, encoded_r])

cos_distance = Reshape((1,))(cos_distance)
cos_similarity = Lambda(lambda x: 1-x)(cos_distance)

siamese_net2 = Model([left_input, right_input], [cos_distance])
siamese_net2.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
siamese_net2.compile(loss="mse",optimizer=optimizer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 8192)]       0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 1200)]       0           []                               
                                                                                                  
 sequential_2 (Sequential)      (None, 1200)         9836400     ['input_4[0][0]']                
                                                                                                  
 sequential_3 (Sequential)      (None, 800)          2830300     ['input_3[0][0]',                
                                                                  'sequential_2[0][0]']     

In [15]:
output = np.concatenate((np.ones(27638), np.zeros(27638)))

siamese_net2.fit(x=[np.concatenate((text_embeddings, np.concatenate((text_embeddings[2:], text_embeddings[0:2])))),np.concatenate((image_embeddings, image_embeddings))],
                y=output, 
                batch_size=16, 
                epochs=20, 
                validation_split=.1)

Epoch 1/20
3110/3110 [==============================] - 262s 84ms/step - loss: 0.2701 - val_loss: 0.2903
Epoch 2/20
3110/3110 [==============================] - 265s 85ms/step - loss: 0.2388 - val_loss: 0.2583
Epoch 3/20
3110/3110 [==============================] - 258s 83ms/step - loss: 0.2310 - val_loss: 0.2281
Epoch 4/20
3110/3110 [==============================] - 261s 84ms/step - loss: 0.2236 - val_loss: 0.1816
Epoch 5/20
3110/3110 [==============================] - 260s 84ms/step - loss: 0.2178 - val_loss: 0.1868
Epoch 6/20
3110/3110 [==============================] - 259s 83ms/step - loss: 0.2128 - val_loss: 0.2622
Epoch 7/20
3110/3110 [==============================] - 259s 83ms/step - loss: 0.2080 - val_loss: 0.1832
Epoch 8/20
3110/3110 [==============================] - 257s 83ms/step - loss: 0.2032 - val_loss: 0.2451
Epoch 9/20
3110/3110 [==============================] - 256s 82ms/step - loss: 0.1996 - val_loss: 0.1994
Epoch 10/20
3110/3110 [==============================] 

In [16]:
siamese_net2.predict([text_embeddings,image_embeddings])

array([[0.5706685 ],
       [0.65702397],
       [0.4776135 ],
       ...,
       [0.37697098],
       [0.50471795],
       [0.3273186 ]], dtype=float32)

In [19]:
f = 1
f2 = 200
siamese_net2.predict([text_embeddings[f:f+10],image_embeddings[f2:f2+10]])

array([[0.5817459 ],
       [0.48907506],
       [0.15846118],
       [0.56743234],
       [0.46045622],
       [0.5991637 ],
       [0.21214887],
       [0.21010935],
       [0.67258435],
       [0.2737526 ]], dtype=float32)

In [53]:
siamese_net2.save('siamese_best.h5')

In [86]:
import random
random.randint(1,27638)

16733

In [97]:
my_pred_dict = {}
for j in range(10):
  random_pic = random.randint(1,27638)
  preds = []
  for i in range(27638): 
    preds.append(siamese_net2.predict([text_embeddings[i].reshape((1,1200)),image_embeddings[random_pic-1].reshape((1,8192))]))

  pic2recipe_df = pd.DataFrame({'id':range(1,27639), 'score':preds})
  my_pred_dict[random_pic] = list(pic2recipe_df.sort_values(by='score').id[-5:])


In [98]:
my_pred_dict

{4878: [15711, 9311, 14621, 9841, 5042],
 6624: [9067, 7828, 9841, 19946, 7532],
 8889: [10288, 27094, 3778, 15809, 15679],
 15218: [6201, 27414, 7133, 18080, 12433],
 15981: [25516, 27503, 14296, 873, 2875],
 17943: [754, 27288, 17767, 12177, 21448],
 20840: [26470, 1592, 6201, 19753, 10695],
 20868: [25470, 21019, 3027, 17452, 20329],
 22011: [20032, 8400, 18963, 25655, 1013],
 23069: [1319, 13281, 177, 7156, 7739]}

In [76]:
taco_pic = pd.DataFrame({'id':range(1,27639), 'score':preds})

In [89]:
list(taco_pic.sort_values(by='score').id[-5:])

[14583, 1317, 5374, 10771, 17213]

In [79]:
taco_pic.iloc[10196]

id                10197
score    [[0.60904324]]
Name: 10196, dtype: object